In [7]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
import numpy as np
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
from qiskit.algorithms.minimum_eigensolvers import AdaptVQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver


In [53]:
def H6_adaptvqe(k,dist):
    
    # Define Molecule
    molecule = MoleculeInfo(
        ["H","H","H","H","H","H"],[(0.0,0.0,0.0),(dist, 0.0, 0.0),(2*dist, 0.0, 0.0),(3*dist, 0.0, 0.0),(4*dist, 0.0, 0.0),(5*dist, 0.0, 0.0)],
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )
    driver = PySCFDriver().from_molecule(molecule)

    problem = driver.run()
    converter = QubitConverter(JordanWignerMapper(),two_qubit_reduction=True)
    transformer = ActiveSpaceTransformer(
    num_electrons=2, #how many electrons we have in our active space
    num_spatial_orbitals=3, #how many orbitals we have in our active space
    )
    
    #problem = transformer.transform(problem=problem)
    print('粒子数{0}，空间轨道数 {1}'.format(problem.num_particles,problem.num_spatial_orbitals))
    
    initial_state = HartreeFock(num_particles = problem.num_particles,num_spatial_orbitals = problem.num_spatial_orbitals,qubit_converter = converter)
    ansatz = UCCSD(num_particles = problem.num_particles,num_spatial_orbitals = problem.num_spatial_orbitals,qubit_converter = converter,initial_state=initial_state)
    vqe = VQE(Estimator(), ansatz, SLSQP())
    vqe.initial_point = np.zeros(ansatz.num_parameters)
    
    adapt_vqe = AdaptVQE(vqe,max_iterations=30,threshold=1e-4)
    adapt_vqe.supports_aux_operators = lambda: True  # temporary fix
    solver = GroundStateEigensolver(converter, adapt_vqe)
    adapt_result = solver.solve(problem)
    print('AdaptVQE 已经完成：{0}%'.format(((k+1)/80)*100))
    return adapt_result

In [55]:
def H6_uccsd(dist,k):
    # Define Molecule
    molecule = MoleculeInfo(
        ["H","H","H","H","H","H"],[(0.0,0.0,0.0),(dist, 0.0, 0.0),(2*dist, 0.0, 0.0),(3*dist, 0.0, 0.0),(4*dist, 0.0, 0.0),(5*dist, 0.0, 0.0)],
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )
    driver = PySCFDriver().from_molecule(molecule)

    problem = driver.run()
    converter = QubitConverter(JordanWignerMapper(),two_qubit_reduction=True)
    transformer = ActiveSpaceTransformer(
    num_electrons=2, #how many electrons we have in our active space
    num_spatial_orbitals=3, #how many orbitals we have in our active space
    )
    #problem = transformer.transform(problem=problem)
    print('粒子数{0}，空间轨道数 {1}'.format(problem.num_particles,problem.num_spatial_orbitals))
    
    initial_state = HartreeFock(num_particles = problem.num_particles,num_spatial_orbitals = problem.num_spatial_orbitals,qubit_converter = converter)
    ansatz = UCCSD(num_particles = problem.num_particles,num_spatial_orbitals = problem.num_spatial_orbitals,qubit_converter = converter,initial_state=initial_state)
    vqe = VQE(Estimator(), ansatz, SLSQP())
    vqe.initial_point = np.zeros(ansatz.num_parameters)
    
    # adapt_vqe = AdaptVQE(vqe)
    # adapt_vqe.supports_aux_operators = lambda: True  # temporary fix
    solver = GroundStateEigensolver(converter, vqe)
    uccsd_result = solver.solve(problem)
    print('UCCSD ansatz 共有 {0} 个参数'.format(ansatz.num_parameters))
    print('UCCSD_VQE 已经完成：{0}%'.format(((k+1)/80)*100))
    
    return uccsd_result

In [56]:
distance = np.linspace(0.5,2.5,80)
adapt_vqe_result = []
uccsd_vqe_result = []

for index,dis in enumerate(distance):
    adapt_vqe_result.append(H6_adaptvqe(dist=dis,k=index))
    uccsd_vqe_result.append(H6_uccsd(dist=dis,k=index))

粒子数(3, 3)，空间轨道数 6


KeyboardInterrupt: 

In [52]:
result = H6_uccsd(0.23,k=1)
result.num_parameters

粒子数(3, 3)，空间轨道数 6


117